# Exploring the Potential of Large Language Models on foreign languages (Arabic)

In this notebook, we will embark on an exciting journey to explore the capabilities of **five large language models** for the **Arabic language**. Our stars for this journey are:

1. **RedPajama:** [togethercomputer/RedPajama-INCITE-Instruct-3B-v1](https://huggingface.co/togethercomputer/RedPajama-INCITE-Instruct-3B-v1)
2. **Dolly V2:** [databricks/dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b)
3. **OPT:** [facebook/opt-2.7b](https://huggingface.co/facebook/opt-2.7b)
4. **GPT Neo 2.7B:** [EleutherAI/gpt-neo-2.7B](https://huggingface.co/EleutherAI/gpt-neo-2.7B)

Our mission is to put these language models to the test on an **Arabic dataset** for the task of generating tags for **Arabic quotes**. To accomplish this, we will employ the innovative **QLoRA** technique to fine-tune each model specifically for our task.

Once the models are fine-tuned, we will gather their results and compare their performance. It will be fascinating to observe how each model tackles the challenge of generating meaningful tags for Arabic quotes.

---

And here's the best part - we will conduct all these experiments using the **free Google Colab notebook**! This means everyone can test and explore the power of these LLMs on Arabic language tasks.

---

# Model: databricks/dolly-v2-3b

In [1]:
# get the running time of each cell
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.3 MB/s eta 0:00:00
time: 329 µs (started: 2023-09-05 15:11:56 +00:00)


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "databricks/dolly-v2-3b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.
    bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

time: 2min 14s (started: 2023-09-05 15:13:13 +00:00)


In [4]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

time: 112 ms (started: 2023-09-05 15:15:28 +00:00)


In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

time: 1.26 ms (started: 2023-09-05 15:15:28 +00:00)


In [6]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],   # line for gpt-neox and Dolly-v2
    #target_modules=["q_proj", "v_proj"], # line for Opt
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 2621440 || all params: 1519416320 || trainable%: 0.172529409187865
time: 9.51 s (started: 2023-09-05 15:15:28 +00:00)


## Importing and pre-processing Data

In [7]:
from datasets import load_dataset

dataset = load_dataset("AhmedBou/Arabic_Quotes")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

time: 4.06 s (started: 2023-09-05 15:15:37 +00:00)


In [8]:
data=dataset

time: 451 µs (started: 2023-09-05 15:15:41 +00:00)


In [9]:
def merge_columns(example):
  example["prediction"] = "quote: " + example["quote"] + " tags: " + example['tags']
  return example

data['train'] = data['train'].map(merge_columns)
data['train']["prediction"][30:50]


Map:   0%|          | 0/5964 [00:00<?, ? examples/s]

["quote: وقائع التاريخ الكبرى عائمات جليد ، طرفها ظاهر فوق الماء ، و كتلتها الرئيسية تحت سطحه ، و من يريد استكشافها عليه أن يغوص . tags: ['تاريخ']",
 "quote: لا أدري بأي منطق يستنكرون استعباد الفرد ويستسيغون استعباد الشعوب. tags: ['حرية']",
 "quote: سأحبك دائما. عندما يكون هذا الشعر الأحمر أبيض ، سأظل أحبك. عندما يتم استبدال نعومة الشباب بنعومة العمر ، سأظل أرغب في لمس بشرتك. عندما يمتلئ وجهك بخطوط كل ابتسامة ابتسمتها على الإطلاق ، من كل مفاجأة رأيتها تومض في عينيك ، عندما تترك كل دمعة بكيت بصماتها على وجهك ، سأعتز بك أكثر ، لأنني كنت هناك لأرى كل شيء. سوف أشارك حياتك معك ، ميريديث ، وسأحبك حتى يخرج النفس الأخير من جسدك أو جسدك . tags: ['ملهمة', 'علاقات']",
 "quote: أصدق حزن .. ابتسامة في عين دامعة . tags: ['خواطر']",
 "quote: لن يحكم أحد فى ملك الله إلا بما أراد الله. tags: ['حقيقة']",
 "quote: عندما لا نتوقع ذلك ، تضعنا الحياة تحديًا لاختبار شجاعتنا واستعدادنا للتغيير ؛ في مثل هذه اللحظة ، لا فائدة من التظاهر بعدم حدوث شيء أو القول إننا لسنا مستعدين بعد. التحدي لن ينتظر. الحياة لا تن

time: 599 ms (started: 2023-09-05 15:15:41 +00:00)


In [10]:
data = data['train'].map(lambda samples: tokenizer(samples["prediction"]), batched=True)

Map:   0%|          | 0/5964 [00:00<?, ? examples/s]

time: 2.04 s (started: 2023-09-05 15:15:42 +00:00)


## Model FineTuning

Transformers Trainer Documentation:

https://huggingface.co/docs/transformers/main_classes/trainer

In [11]:
import transformers

# tokenizer.pad_token needed for Opt, gpt-neo-x, and RedPajama tokenizer
#tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=4,
        max_steps=20,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.083600
2,3.270400
3,3.341800
4,2.981900
5,2.453300
6,2.132600
7,2.619600
8,2.211900
9,2.062200
10,2.463400


TrainOutput(global_step=20, training_loss=2.388903135061264, metrics={'train_runtime': 55.6897, 'train_samples_per_second': 1.437, 'train_steps_per_second': 0.359, 'total_flos': 55422157271040.0, 'train_loss': 2.388903135061264, 'epoch': 0.01})

time: 57.2 s (started: 2023-09-05 15:15:44 +00:00)


## Inference

arabic quotes tags https://arabic-quotes.com/

In [12]:
list_10_Quotes = ["عندما تعرض عليك مشكلة ابعد نفسك عن التحيز و الافكار المسبقة .. و تعرف على حقائق الموقف ، و رتبها، ثم اتخذ الموقف الذي يظهر لك انه أكثر عدلا و تمسك به .",
                  "من حلاوة ما ذقته فى القرآن .. أريد أن أنقل هذه الحلاوة للناس .",
                  "الدنيا كالماء المالح كلما ازددت شرباً منها ازددت عطشاً .",
                  "لا تعبدوا الله ليعطي، بل اعبدوه ليرضى، فإن رضي أدهشكم بعطائه.",
                  "قبل أن يتكلم الرجل الحكيم عليه إن يفكر تماماً فيما يقول ، و فيمن يستمـع إليه ، و أين و متى يتـكلم .",
                  "قبل اقدامك على معرفة الحقيقة ، تذكر جيداً أنك بعد اكتشافها لن تستطيع العودة الى ما كنت عليه سابقاً .",
                  "لن يحكم أحد فى ملك الله إلا بما أراد الله.",
                  "الحياة طفل ينبغي ملاطفته حتى ينام .",
                  "لا يكون الانسان حرا إلا عندما يود أن يكون.",
                  "قم بكل ما تفعله في حياتك وكأنه آخر عمل تقوم به."]

list_10_tags =[["العدل", "حل المشكلات", "الافكار المسبقة"],
                ["القرآن", "الحلاوة", "نقل الحلاوة للناس"],
                ["الدنيا", "الماء المالح", "العطش"],
                ["العبادة", "إرضاء الله", "العطاء"],
                ["الرجل الحكيم", "التفكير قبل الكلام", "توقيت الكلام"],
                ["اكتشاف الحقيقة", "التغييرات اللاعودة", "عدم الرجوع للماضي"],
                ["حكم الله", "مشيئة الله", "القضاء والقدر"],
                ["الحياة كطفل", "الملاطفة", "النوم"],
                ["حرية الإنسان", "الحرية بالإرادة", "التمني بالحرية"],
                ["الإلتزام بالأعمال", "الأعمال الأخيرة", "النهاية"]]


time: 581 µs (started: 2023-09-05 15:16:41 +00:00)


In [13]:
list_10_inputs = ["quote: عندما تعرض عليك مشكلة ابعد نفسك عن التحيز و الافكار المسبقة .. و تعرف على حقائق الموقف ، و رتبها، ثم اتخذ الموقف الذي يظهر لك انه أكثر عدلا و تمسك به .  tags:",
                  "quote: من حلاوة ما ذقته فى القرآن .. أريد أن أنقل هذه الحلاوة للناس .  tags:",
                  "quote: الدنيا كالماء المالح كلما ازددت شرباً منها ازددت عطشاً .   tags:",
                  "quote: لا تعبدوا الله ليعطي، بل اعبدوه ليرضى، فإن رضي أدهشكم بعطائه.   tags:",
                  "quote: قبل أن يتكلم الرجل الحكيم عليه إن يفكر تماماً فيما يقول ، و فيمن يستمـع إليه ، و أين و متى يتـكلم .   tags:",
                  "quote: قبل اقدامك على معرفة الحقيقة ، تذكر جيداً أنك بعد اكتشافها لن تستطيع العودة الى ما كنت عليه سابقاً .   tags:",
                  "quote: لن يحكم أحد فى ملك الله إلا بما أراد الله.   tags:",
                  "quote: الحياة طفل ينبغي ملاطفته حتى ينام .   tags:",
                  "quote: لا يكون الانسان حرا إلا عندما يود أن يكون.  tags:",
                  "quote: قم بكل ما تفعله في حياتك وكأنه آخر عمل تقوم به.   tags:"]

time: 434 µs (started: 2023-09-05 15:16:41 +00:00)


In [14]:
%%time
res_dolly_v2_3b = []
device = "cuda:0"
for elm in list_10_inputs:
  inputs = tokenizer(elm, return_tensors="pt").to(device)
  outputs = model.generate(**inputs, max_new_tokens=10)
  generated_text=tokenizer.decode(outputs[0], skip_special_tokens=True)
  res_dolly_v2_3b.append(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_i

CPU times: user 30.1 s, sys: 58.9 ms, total: 30.2 s
Wall time: 32.1 s
time: 32.1 s (started: 2023-09-05 15:16:41 +00:00)


## Generated Tags

In [15]:
res_dolly_v2_3b

["quote: عندما تعرض عليك مشكلة ابعد نفسك عن التحيز و الافكار المسبقة.. و تعرف على حقائق الموقف ، و رتبها، ثم اتخذ الموقف الذي يظهر لك انه أكثر عدلا و تمسك به.  tags: ['التحديات']. tags",
 "quote: من حلاوة ما ذقته فى القرآن.. أريد أن أنقل هذه الحلاوة للناس.  tags: ['أحداث'].. tags",
 "quote: الدنيا كالماء المالح كلما ازددت شرباً منها ازددت عطشاً.   tags: ['الدنيا']. tags:",
 "quote: لا تعبدوا الله ليعطي، بل اعبدوه ليرضى، فإن رضي أدهشكم بعطائه.   tags: ['تعليم'] ['تع",
 "quote: قبل أن يتكلم الرجل الحكيم عليه إن يفكر تماماً فيما يقول ، و فيمن يستمـع إليه ، و أين و متى يتـكلم.   tags: ['الأمثل']. tags:",
 "quote: قبل اقدامك على معرفة الحقيقة ، تذكر جيداً أنك بعد اكتشافها لن تستطيع العودة الى ما كنت عليه سابقاً.   tags: ['تذكر']. tags: ['",
 "quote: لن يحكم أحد فى ملك الله إلا بما أراد الله.   tags: ['تعليم']. tags:",
 "quote: الحياة طفل ينبغي ملاطفته حتى ينام.   tags: ['أنت']. tags: ['أ",
 "quote: لا يكون الانسان حرا إلا عندما يود أن يكون.  tags: ['تعليم']. tags:",
 "quote: قم بكل ما تفعله

time: 3.81 ms (started: 2023-09-05 15:17:14 +00:00)


## Clean generated tags

In [16]:
import re

def output_parser(sentences):
    result_list = []
    tags_start = "tags:"
    for sentence in sentences:
        if tags_start in sentence:
            tags_str = sentence.partition(tags_start)[2].strip()
            # Extract the sentence before "tags list" and remove "tags list" from the original sentence
            quote = sentence.partition(tags_start)[0].replace(tags_start, '').strip()
            # Removing the square brackets and quotation marks
            tags_str = tags_str.replace("[", "").replace("]", "").replace("،", "").replace('"', '').strip()
            # Extracting individual tags and filtering out non-Arabic words
            tags = re.findall(r'\b[\u0600-\u06FF]+\b', tags_str)
            result_list.append((quote, tags))
    return result_list

time: 907 µs (started: 2023-09-05 15:17:14 +00:00)


In [17]:
results = output_parser(res_dolly_v2_3b)
generated_tags=[]
for quote, tags in results:
    generated_tags.append(tags)
    print(f"{quote}")
    print(f"RedPajama Generated Tags: {tags}\n")

quote: عندما تعرض عليك مشكلة ابعد نفسك عن التحيز و الافكار المسبقة.. و تعرف على حقائق الموقف ، و رتبها، ثم اتخذ الموقف الذي يظهر لك انه أكثر عدلا و تمسك به.
RedPajama Generated Tags: ['التحديات']

quote: من حلاوة ما ذقته فى القرآن.. أريد أن أنقل هذه الحلاوة للناس.
RedPajama Generated Tags: ['أحداث']

quote: الدنيا كالماء المالح كلما ازددت شرباً منها ازددت عطشاً.
RedPajama Generated Tags: ['الدنيا']

quote: لا تعبدوا الله ليعطي، بل اعبدوه ليرضى، فإن رضي أدهشكم بعطائه.
RedPajama Generated Tags: ['تعليم', 'تع']

quote: قبل أن يتكلم الرجل الحكيم عليه إن يفكر تماماً فيما يقول ، و فيمن يستمـع إليه ، و أين و متى يتـكلم.
RedPajama Generated Tags: ['الأمثل']

quote: قبل اقدامك على معرفة الحقيقة ، تذكر جيداً أنك بعد اكتشافها لن تستطيع العودة الى ما كنت عليه سابقاً.
RedPajama Generated Tags: ['تذكر']

quote: لن يحكم أحد فى ملك الله إلا بما أراد الله.
RedPajama Generated Tags: ['تعليم']

quote: الحياة طفل ينبغي ملاطفته حتى ينام.
RedPajama Generated Tags: ['أنت', 'أ']

quote: لا يكون الانسان حرا إل

## Accuracy Score

In [18]:
def get_arabic_string_similarity(list1, list2):
    def preprocess_arabic_string(string):
        # Preprocess the Arabic string by removing duplicate characters and spaces
        return set(string.replace(" ", ""))

    set1 = preprocess_arabic_string("".join(list1))
    set2 = preprocess_arabic_string("".join(list2))

    # Calculate the intersection and union of the two sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    # Calculate the Jaccard similarity
    similarity_score = len(intersection) / len(union)

    # Normalize the similarity score with respect to the maximum possible score
    max_score = 1.0
    normalized_score = similarity_score / max_score

    return normalized_score

time: 658 µs (started: 2023-09-05 15:17:14 +00:00)


In [19]:
def acc_score(generated_tags):
  scores=[]
  for elm in range(len(list_10_tags)):
    similarity_score = get_arabic_string_similarity(list_10_tags[elm], generated_tags[elm])
    print(similarity_score)
    scores.append(similarity_score)
  # Calculate the sum of all elements in the list
  total = sum(scores)
  # Calculate the average by dividing the sum by the number of elements in the list
  score = total / len(scores)
  return score

time: 562 µs (started: 2023-09-05 15:17:14 +00:00)


In [20]:
score = acc_score(generated_tags)

0.3125
0.15384615384615385
0.45454545454545453
0.13333333333333333
0.2
0.15789473684210525
0.16666666666666666
0.07692307692307693
0.2857142857142857
0.3333333333333333
time: 853 µs (started: 2023-09-05 15:17:14 +00:00)


In [21]:
print(f"score: {score:.2f}")

score: 0.23
time: 489 µs (started: 2023-09-05 15:17:14 +00:00)


### Model hosting on Huggingface Hub

In [22]:
from huggingface_hub import notebook_login
#Use your personal HugginFace Token, it's available for free in your HugginFace account
notebook_login()

time: 42.2 ms (started: 2023-09-05 15:17:14 +00:00)


In [24]:
# Give the link for your HuggingFace repository and Name your model
model.push_to_hub("AhmedBou/databricks-dolly-v2-3b_Arabic_Tags",
                  use_auth_token=True,
                  commit_message="basic training",
                  #private=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/10.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AhmedBou/databricks-dolly-v2-3b_Arabic_Tags/commit/fc206ab09915f773b73051445950b44818ec039e', commit_message='basic training', commit_description='', oid='fc206ab09915f773b73051445950b44818ec039e', pr_url=None, pr_revision=None, pr_num=None)

time: 1.56 s (started: 2023-09-05 15:18:41 +00:00)


## Using Our Hosted Model

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "AhmedBou/databricks-dolly-v2-3b_Arabic_Tags"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
batch = tokenizer("“Multiple Regression for Appraisal” -->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))